In [ ]:
import numpy as np
import numpy.random as rand
%matplotlib inline
import matplotlib.pyplot as plt

#Next we are going to import some specific libraries we will use to get the animation to work cleanly
from IPython.display import display, clear_output
import time  

# function plotgrid() takes in a 2D array and uses pyplot to make a plot.
# this function returns no values!

def plotgrid(myarray):
    
    # first create two vectors based on the x and y sizes of the grid
    x_range = np.linspace(0, myarray.shape[0], myarray.shape[0]) 
    y_range = np.linspace(0, myarray.shape[0], myarray.shape[0])
    
    # use the numpy meshgrid function to create two matrices 
    # of the same size as myarray with x and y indexes
    x_indexes, y_indexes = np.meshgrid(x_range, y_range)
    
    # make a list of all the x and y indexes that are either squares or triangles.
    # the notation below is relatively new to us; it means that when myarray==(value),
    # only record those values.
    sq_x = x_indexes[myarray == 1]; 
    sq_y = y_indexes[myarray == 1]; 
    tr_x = x_indexes[myarray == 2]; 
    tr_y = y_indexes[myarray == 2]; 
    
    # plot the squares and triangles.  make the size of the polygons 
    # larger than the default so they're easy to see!
    plt.plot(sq_x,sq_y, 'gs',markersize=10)   
    plt.plot(tr_x,tr_y, 'rs',markersize=10)  
    
    #Set the x and y limits to include half a space overlap so we don't cut off the shapes
    plt.ylim([-1,myarray.shape[0]+1]) 
    plt.xlim([-1,myarray.shape[0]+1])


def set_board(board_size=20,prob_tree=0.25,prob_fire=0.01):

    game_board = np.zeros((board_size,board_size),dtype='int64')
    
    for i in range(board_size):
        for j in range(board_size):
            if rand.random() < prob_tree:
                game_board[i,j] = 1
            #if rand.random() < prob_fire:
            #    game_board[i,j] = 2
    
    game_board[0,:] = 2
    
    return game_board

def advance_board(game_board, prob_lightning=0.0,prob_immune=0.0):
    
    new_board = np.zeros_like(game_board)
    
    for i in range(game_board.shape[0]):
        for j in range(game_board.shape[1]):
    
            if game_board[i,j] == 0 or game_board[i,j] == 2:
                new_board[i,j] = 0
    
            if game_board[i,j] == 1:
                new_board[i,j] = 1

                if rand.random() < prob_lightning and rand.random() >= prob_immune:
                    new_board[i,j] = 2

                if i > 0:
                    if game_board[i-1,j] == 2 and rand.random() >= prob_immune:
                        new_board[i,j] = 2
                    
                if j > 0:
                    if game_board[i,j-1] == 2 and rand.random() >= prob_immune:
                        new_board[i,j] = 2

                if i < game_board.shape[0]-1:
                    if game_board[i+1,j] == 2 and rand.random() >= prob_immune:
                        new_board[i,j] = 2
                        
                if j < game_board.shape[1]-1:                    
                    if game_board[i,j+1] == 2 and rand.random() >= prob_immune:
                        new_board[i,j] = 2
                    
    return new_board

def calc_stats(game_board):
    
    frac_empty = (game_board == 0).sum() / game_board.size
    frac_tree = (game_board == 1).sum() / game_board.size
    frac_fire = (game_board == 2).sum() / game_board.size
    
    return frac_empty, frac_tree, frac_fire


In [ ]:
board_size = 40
turns = 10
prob_tree=0.75
prob_fire=0.005

fig = plt.figure(figsize=(6,6))

game_board = set_board(board_size=board_size, prob_tree=prob_tree, prob_fire=prob_fire)

plotgrid(game_board)

on_fire = True
while on_fire == True:
    game_board = advance_board(game_board)
    
    plotgrid(game_board)
    
    # Animation part (dosn't change)
    time.sleep(0.5)         # Sleep for half a second to slow down the animation
    clear_output(wait=True) # Clear output for dynamic display
    display(fig)            # Reset display
    fig.clear()             # Prevent overlapping and layered plots

    frac_empty, frac_trees, frac_fire = calc_stats(game_board)
    if frac_fire == 0:
        on_fire = False

plt.close()                 # Close dynamic display


In [ ]:
board_size = 40
prob_fire=0.005

samples = 10

f_tree = []
f_burned = []

for tree_fraction in np.arange(0.01,1.01,0.01):

    f_burned_accum = 0.0
    
    print("tree fraction is:", tree_fraction)
    
    for i in range(samples):
        game_board = set_board(board_size=board_size, prob_tree=tree_fraction, prob_fire=prob_fire)

        on_fire = True
        while on_fire == True:
            game_board = advance_board(game_board)
            frac_empty, frac_trees, frac_fire = calc_stats(game_board)
            if frac_fire == 0:
                on_fire = False
        f_burned_accum += frac_empty - (1.0-tree_fraction)
    
    f_tree.append(tree_fraction)
    f_burned.append(f_burned_accum/samples)

plt.plot(f_tree, f_burned)
plt.xlabel("tree fraction")
plt.ylabel("burned fraction (normalized)")


In [ ]:
np.arange(0,1,0.2)